In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np

In [2]:
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle

In [3]:
top_200 = pd.read_csv("top_200.csv")

In [4]:
top_200

,Title,Artist
0,Lil Boo Thang,Paul Russell
1,Wrap Me Up,Jimmy Fallon & Meghan Trainor
2,Lovin On Me,Jack Harlow
3,Find Your People,Drew Holcomb & The Neighbors
4,DJ Play A Christmas Song,Cher
...,...,...
172,Mi Ex Tenia Razon,Karol G
173,Different 'Round Here,Riley Green Featuring Luke Combs
174,But I Got A Beer In My Hand,Luke Bryan
175,Better Than Ever,YoungBoy Never Broke Again & Rod Wave


In [5]:
top_200['Title'] = top_200['Title'].apply(str.lower)

In [6]:
top_200

,Title,Artist
0,lil boo thang,Paul Russell
1,wrap me up,Jimmy Fallon & Meghan Trainor
2,lovin on me,Jack Harlow
3,find your people,Drew Holcomb & The Neighbors
4,dj play a christmas song,Cher
...,...,...
172,mi ex tenia razon,Karol G
173,different 'round here,Riley Green Featuring Luke Combs
174,but i got a beer in my hand,Luke Bryan
175,better than ever,YoungBoy Never Broke Again & Rod Wave


In [7]:
 top_200['Title'].values

array(['lil boo thang', 'wrap me up', 'lovin on me', 'find your people',
       'dj play a christmas song',
       'i remember everything (feat. kacey musgraves)', 'white horse',
       'save me (with lainey wilson)', 'need a favor',
       'standing next to you', 'houdini', 'cruel summer',
       '90s rap mashup', 'lose control', '3d',
       'all i want for christmas is you', "you're the inspiration",
       'christmas / sarajevo 12/24 (instrumental)',
       'standing next to you (band version)', 'thinkin’ bout me',
       'standing next to you (future funk remix)',
       'fairytale of philadelphia', 'where the wild things are',
       'standing next to you (pbr&b remix)', 'daylight',
       'standing next to you (slow jam remix)',
       'standing next to you (holiday remix)',
       'standing next to you (latin trap remix)',
       'standing next to you (instrumental)', 'paint the town red',
       'fast car', "rockin' around the christmas tree (single)",
       'last night', 'wa

In [8]:
clustered_audio_feat = pd.read_csv("clustered_audio_feat.csv")

In [9]:
clustered_audio_feat

,track.name,name,song_id,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,clusters_1
0,Like a Rolling Stone,Bob Dylan,3AhXZa8sUQht0UEdBJgpGc,0.482,0.721,0,-6.839,1,0.0321,0.731000,...,0.557,95.263,audio_features,3AhXZa8sUQht0UEdBJgpGc,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4,6
1,Smells Like Teen Spirit,Nirvana,3oTlkzk1OtrhH8wBAduVEi,0.485,0.863,1,-9.027,1,0.0495,0.000012,...,0.767,116.835,audio_features,3oTlkzk1OtrhH8wBAduVEi,spotify:track:3oTlkzk1OtrhH8wBAduVEi,https://api.spotify.com/v1/tracks/3oTlkzk1Otrh...,https://api.spotify.com/v1/audio-analysis/3oTl...,300977,4,1
2,A Day In The Life - Remastered,The Beatles,3ZFBeIyP41HhnALjxWy1pR,0.364,0.457,4,-14.162,0,0.0675,0.290000,...,0.175,163.219,audio_features,3ZFBeIyP41HhnALjxWy1pR,spotify:track:3ZFBeIyP41HhnALjxWy1pR,https://api.spotify.com/v1/tracks/3ZFBeIyP41Hh...,https://api.spotify.com/v1/audio-analysis/3ZFB...,337413,4,16
3,Good Vibrations (Mono),The Beach Boys,5Qt4Cc66g24QWwGP3YYV9y,0.398,0.413,1,-10.934,1,0.0388,0.082200,...,0.331,133.574,audio_features,5Qt4Cc66g24QWwGP3YYV9y,spotify:track:5Qt4Cc66g24QWwGP3YYV9y,https://api.spotify.com/v1/tracks/5Qt4Cc66g24Q...,https://api.spotify.com/v1/audio-analysis/5Qt4...,219147,4,6
4,Johnny B Goode,Chuck Berry,7MH2ZclofPlTrZOkPzZKhK,0.518,0.756,10,-10.851,1,0.0915,0.735000,...,0.968,166.429,audio_features,7MH2ZclofPlTrZOkPzZKhK,spotify:track:7MH2ZclofPlTrZOkPzZKhK,https://api.spotify.com/v1/tracks/7MH2ZclofPlT...,https://api.spotify.com/v1/audio-analysis/7MH2...,160893,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48581,Blue feat. Ilse de Lange,Zucchero,583qzYY2ARvSDchyq8PkTF,0.624,0.767,9,-4.902,1,0.0317,0.073400,...,0.443,158.124,audio_features,583qzYY2ARvSDchyq8PkTF,spotify:track:583qzYY2ARvSDchyq8PkTF,https://api.spotify.com/v1/tracks/583qzYY2ARvS...,https://api.spotify.com/v1/audio-analysis/583q...,288693,4,3
48582,Blue feat. Ilse de Lange,Ilse DeLange,583qzYY2ARvSDchyq8PkTF,0.624,0.767,9,-4.902,1,0.0317,0.073400,...,0.443,158.124,audio_features,583qzYY2ARvSDchyq8PkTF,spotify:track:583qzYY2ARvSDchyq8PkTF,https://api.spotify.com/v1/tracks/583qzYY2ARvS...,https://api.spotify.com/v1/audio-analysis/583q...,288693,4,3
48583,"All For Love - From ""The Three Musketeers"" Sou...",Bryan Adams,3sDWjYgYzAPUcwzuo6OGs8,0.462,0.548,2,-6.938,1,0.0276,0.184000,...,0.127,149.905,audio_features,3sDWjYgYzAPUcwzuo6OGs8,spotify:track:3sDWjYgYzAPUcwzuo6OGs8,https://api.spotify.com/v1/tracks/3sDWjYgYzAPU...,https://api.spotify.com/v1/audio-analysis/3sDW...,282773,4,6
48584,"All For Love - From ""The Three Musketeers"" Sou...",Sting,3sDWjYgYzAPUcwzuo6OGs8,0.462,0.548,2,-6.938,1,0.0276,0.184000,...,0.127,149.905,audio_features,3sDWjYgYzAPUcwzuo6OGs8,spotify:track:3sDWjYgYzAPUcwzuo6OGs8,https://api.spotify.com/v1/tracks/3sDWjYgYzAPU...,https://api.spotify.com/v1/audio-analysis/3sDW...,282773,4,6


In [10]:
scaler = pickle.load(open("scaler.pkl","rb"))

In [11]:
kmeans = pickle.load(open("kmeans.pkl","rb"))

In [12]:
secrets_file = open("secrets.txt","r")

In [13]:
string = secrets_file.read()

In [14]:
string

'clientid:41b65b653f8548c1815431d54a305990\nclientsecret:918beac42fd848b58c000da5a853fbde'

In [15]:
string.split('\n')

['clientid:41b65b653f8548c1815431d54a305990',
 'clientsecret:918beac42fd848b58c000da5a853fbde']

In [16]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [17]:
secrets_dict

{'clientid': '41b65b653f8548c1815431d54a305990',
 'clientsecret': '918beac42fd848b58c000da5a853fbde'}

# Connection to spotify

In [18]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [19]:
#def recommend_top_200(song,artist):
    
    #This functions takes an artist name and song title as arguments and looks for a different song on the top 200 list
    
    #filtered_df = top_200[top_200['Title'] != song]
    #random_song = filtered_df.sample(1)
    #recommended_title = random_song['Title'].values
    #recommended_artist = random_song['Artist'].values
    #print(f"Entered song '{song}'. Here's a recommendation:")
    #print(f"Title: {recommended_title}, Artist: {recommended_artist}")

In [20]:
#recommend_top_200("a little less conversation", "Elvis Presley")

In [21]:
def recommend_song_test():
    input_song = input('Enter the title of a song: ').lower()
    input_artist = input('Enter the artist: ').lower()
    print('\n')
    if input_song in top_200['Title'].values:
        recommend_top_200(input_song,input_artist)
    else:
        print('I do not know that song')

In [ ]:
recommend_song_test()

In [ ]:
def get_info_from_spotify(song,sp):
    '''
    This function takes a song title as argument and a spotipy connection and searched for the song_id
    '''
    results = sp.search(q='track:' + song, type='track')
    if results['tracks']['items']:
        song_id = results['tracks']['items'][0]['id']
    else:
        song_id = None
    return song_id
    

In [ ]:
get_info_from_spotify("p09jg",sp)

In [ ]:
def recommend_song(top_200):
    input_song = input('Enter the title of a song: ').lower()
    input_artist = input('Enter the artist: ').lower()
    print('\n')
    if input_song in top_200['Title'].values:
        recommended_top_200(input_song,input_artist)
    else:
        song_id = get_info_from_spotify(input_song,sp)
        if song_id:
            input_features = sp.audio_features(song_id)
            input_df = pd.DataFrame(input_features)
            selected_features = input_df.select_dtypes(include=np.number)
            selected_features_df = pd.DataFrame(selected_features, columns=selected_features.columns)
            scaler = pickle.load(open('scaler.pkl', 'rb'))
            kmeans = pickle.load(open('kmeans.pkl', 'rb'))
            scaled_features = scaler.transform(selected_features_df)
            predicted_cluster = kmeans.predict(scaled_features)
            cluster_df = clustered_audio_feat[clustered_audio_feat['clusters_1'] == predicted_cluster[0]]
            recommended_song2 = cluster_df.sample(1)
            recommended_title2 = recommended_song2['track.name'].values
            recommended_artist2 = recommended_song2['name'].values
            print(f"Entered song '{input_song}'. Here's a recommendation from clustered_audio_feat:")
            print(f"Title: {recommended_title2}, Artist: {recommended_artist2}")
        else:
            print(f"Sorry, no song found with title '{input_song}'.")
        

In [ ]:
scaler = pickle.load(open('scaler.pkl', 'rb'))


In [ ]:
scaler

kmeans = pickle.load(open('kmeans.pkl', 'rb'))

In [ ]:
kmeans


In [ ]:
recommend_song(top_200)

In [ ]:
#def recommend_song(top_200):
    #input_song = input('Enter the title of a song: ').lower()
    #input_artist = input('Enter the artist: ').lower()
    #print('\n')
    
    # Check if both title and artist match in the DataFrame using mask
    #mask = (top_200['Title'].str.lower() == input_song) & (top_200['Artist'].str.lower() == input_artist)
    
    #if mask.any():
        #print(f"Entered song '{input_song}' by '{input_artist}'. Here's a recommendation:")
        # Filtering out the entered song
        #filtered_df = top_200[~mask]
        
        # Randomly select a song from the filtered DataFrame
        #random_song = filtered_df.sample(1)
        
        # Get the recommended title and artist
        #recommended_title = random_song['Title'].values[0]
        #recommended_artist = random_song['Artist'].values[0]
        #print(f"Title: {recommended_title}, Artist: {recommended_artist}")
        #print(f"Song '{input_song}' by '{input_artist}' not found in the top 200.")
        
    #else:
        #open spotify connection
        #sp = open_spotify_connection()
        #Querying Spotify
        #results = sp.search(q=f"track:{user_input_song_lower}", type="track", limit=1)
        #new_song = results['tracks']['items']
        #new_artist = results['tracks']['items'][0]['id']
        #feature = features(new_song, new_artist)
        #column = list{feature}[0].keys
        #values = [list{feature}[0].values()]
        #df_new_song = pd.DataFrame(data = feature, columns = column)
        #df_new_song = df_new_song.drop(["type","id","uri","track_href","analysis_url","time_signature"],axis=1)
        #Using saved pickle
        #std_new_song = scaler.transform(df_new_song)
        #new_cluster = kmeans.predict(std_new_song)
        #print(new_cluster)
        #df_cluster = clustered_audio_feat[clustered_audio_feat["cluster"] == list(new_cluster)[0]]
        #print(" Recommendation: \n\",random.choice(list(df_cluster))) #df_cluster.sample(1)


In [ ]:
#def recommend_song(top_200):
    #input_song = input('Enter the title of a song: ').lower()
    #input_artist = input('Enter the artist: ').lower()
    #print('\n')
    #Checking if input_song is in top_200 in the column "Title" and filtering so it does not get picked again as top song in the
    # in the recomendation process
    #if input_song in top_200['Title'].values:
        #filtered_df = top_200[top_200['Title'] != input_song]
        #random_song = filtered_df.sample(1)
        #recommended_title = random_song['Title'].values
        #recommended_artist = random_song['Artist'].values
        #print(f"Entered song '{input_song}'. Here's a recommendation:")
        #print(f"Title: {recommended_title}, Artist: {recommended_artist}")
    #else:
        #open spotify connection
        #sp = open_spotify_connection()
        #Querying Spotify
        #results = sp.search(q=f"track:{user_input_song_lower}", type="track", limit=1)
        #feature = features(new_song, new_artist)
        #column = list{feature}[0].keys
        #values = [list{feature}[0].values()]
        #df_new_song = pd.DataFrame(data = feature, columns = column)
        #df_new_song = df_new_song.drop(["type","id","uri","track_href","analysis_url","time_signature"],axis=1)
        #Using saved pickle
        #std_new_song = scaler.transform(df_new_song)
        #new_cluster = kmeans.predict(std_new_song)
        #print(new_cluster)
        #df_cluster = clustered_audio_feat[clustered_audio_feat["cluster_1"] == list(new_cluster)[0]]
        #print(" Recommendation: \n\n",random.choice(list(df_cluster["cluster"])))